In [ ]:
from mpi4py import MPI
import plannerMethods
import time
from repast4py import context as ctx
import repast4py 
from repast4py import parameters
from repast4py import schedule
from repast4py import core
from math import ceil
from typing import Tuple, List, Dict
import numpy as np
import pandas as pd
import pickle
import csv
import os
import sys
import yaml
import random
import abc

from agents.stockItems.Deposit import Deposit
from agents.stockItems.Reserve import Reserve
from agents.stockItems.Loan import Loan

from agents.CentralBank import CentralBank
from agents.CommercialBank import CommercialBank
from agents.AgentReporter import AgentReporter
from agents.CentralPlanner import CentralPlanner

from markets.CreditMarket import CreditMarket
from markets.MarketReporter import MarketReporter



In [ ]:
comm = MPI.COMM_WORLD
rank    = comm.Get_rank()
rankNum = comm.Get_size()


startTime=-1
def T():
    global startTime
    if startTime < 0:
        startTime=time.time()
    return time.time() - startTime
T() #launches the timer

#cpuTimer Tc()
startCpuTime=-1
def Tc():
    global startCpuTime
    if startCpuTime < 0:
        startCpuTime=time.process_time()
    return time.process_time() - startCpuTime
Tc() #launches the cpu timer



model_params = parameters.init_params("./config/model/model1.yaml", "")
agnet_params = parameters.init_params("./config/agent/ini_agent.yaml", "")
params = {**model_params, **agnet_params}

repast4py.random.init(rng_seed=params['myRandom.seed'][rank]) #each rank has a seed
rng = repast4py.random.default_rng 

if os.path.isdir(params["log_file_root"]+"."+str(rank)):
    os.system("rm -R "+params["log_file_root"]+"."+str(rank))
os.makedirs(params["log_file_root"]+"."+str(rank))

#copy in the output folder the starting set of parameters
os.system("cp model1.yaml "+params["log_file_root"]+"."+str(rank)+"/")
os.system("cp firm-features.csv "+params["log_file_root"]+"."+str(rank)+"/")
os.system("cp plannerMethods.py "+params["log_file_root"]+"."+str(rank)+"/")

if rank==0:
    i=0
    while os.path.isdir(params["log_file_root"]+"."+str(rankNum+i)):
        os.system("rm -R "+params["log_file_root"]+"."+str(rankNum+i))
        i+=1
    

    
#moves to the right folder (that you must create and initialize with a firm-features.csv file)
if not os.path.isdir(params["log_file_root"]+"."+str(rank)):
    print("There is no "+params["log_file_root"]+"."+str(rank) + " starting folder!")  
    sys.exit(0)
else: os.chdir(params["log_file_root"]+"."+str(rank))


In [ ]:
class GlobalContainer:
    def __init__(self, params):
        
        '''
        {agent_type: 
            rankid: real agent uid for this type}

        '''

        self.real_agents = dict()
        self.all_agents = dict()
        self.counter = dict()

        if params['agent_distribute_mode'] == 'mean':
            for agent_type, agent_config in params['agents'].items():
                self.real_agents[agent_type] = dict()
                self.all_agents[agent_type] = dict()
                total_amount = agent_config['total_amount']
                groups_amount = self.calculate_group_amounts(agent_config['groups'], total_amount, agent_type)
                
                id_count = 0
                
                for i_rank in range(rankNum):
                    self.real_agents[agent_type][i_rank] = []
                    self.all_agents[agent_type][i_rank] = []
                
                for group_name, group_config in agent_config['groups'].items():
                    amount = groups_amount[group_name]
                    for _ in range(amount):
                        rankID = id_count % rankNum
                        agent_tuple = ((id_count // rankNum, agent_type, rankID),
                                   (group_config['paramGroup'], group_config['isGlobal']))
                    
                        
                        self.real_agents[agent_type][rankID].append(agent_tuple)
                        
                        if group_config['isGlobal']:
                            for i_rank in range(rankNum):
                                self.all_agents[agent_type][i_rank].append(agent_tuple)
                        else:
                            self.all_agents[agent_type][rankID].append(agent_tuple)
                        
                        id_count += 1
                

        else:
            raise ValueError('only mean in current step')

    def calculate_group_amounts(self, groups, total_amount, agent_type):
        group_amounts = {}
        total_share = sum(group['share'] for group in groups.values())

        if total_share != 1:
            print(f'the total share of agent type {agent_type} not equal to 1, we will rebalance the share to 1')

        for group_name, group_config in groups.items():
            share = group_config['share']/total_share
            amount = int(total_amount * share)
            group_amounts[group_name] = amount

        return group_amounts
    
    def getAllAgents(self, agent_type=None, rankId=None, uid_only=True):
        assert (rankId is not None) or (agent_type is not None)

        agentList = []

        if rankId is None:
            # If only agent_type is provided, return agents by type
            for agents in self.all_agents.get(agent_type, {}).values():
                agentList += agents

        elif agent_type is None:
            # If only rankID is provided, return agents by rank
            for agent_type in self.all_agents.keys:
                agentList += self.real_agents[agent_type].get(rankId, [])
        
        else:
            # If both rankID and agent_type are provided, return agents by rank and type
            agentList = self.all_agents[agent_type].get(rankId, [])
        
        if uid_only and agent_type != []:
            agentList = [agents[0] for agents in agentList]
            
        return agentList

    def getRealAgents(self, agent_type=None, rankId=None, uid_only=True):
        assert (rankId is not None) or (agent_type is not None)

        agentList = []

        if rankId is None:
            # If only agent_type is provided, return agents by type
            for agents in self.real_agents.get(agent_type, {}).values():
                agentList += agents

        elif agent_type is None:
            # If only rankID is provided, return agents by rank
            for agent_type in self.real_agents.keys:
                agentList += self.real_agents[agent_type].get(rankId, [])

        else:
            # If both rankID and agent_type are provided, return agents by rank and type
            agentList = self.real_agents[agent_type].get(rankId, [])
        
        if uid_only and agent_type != []:
            agentList = [agents[0] for agents in agentList]
            
        return agentList

    def getRandomRealAgent(self, agent_type=None, rankId=None, uid_only=True):
        assert (rankId is not None) or (agent_type is not None)
        
        agentList = self.getRealAgents(agent_type, rankId, uid_only)
        
        return random.choice(agentList)
    
    def getRandomAllAgent(self, agent_type=None, rankId=None, uid_only=True):
        assert (rankId is not None) or (agent_type is not None)
        
        agentList = self.getAllAgents(agent_type, rankId, uid_only)
    
        return random.choice(agentList)

In [ ]:
def any(iterable):
    for element in iterable:
        if element != 0:
            return True
    return False
    
    
class Firm(core.Agent):
    
    def __init__(self, local_id: int, rank: int, labor:int, capital:float, minOrderDuration:int,\
                 maxOrderDuration:int, recipe: float, laborProductivity: float, maxOrderProduction: float,\
                 assetsUsefulLife: float, plannedMarkup: float, orderObservationFrequency: int, productionType: int,\
                 sectorialClass: int):
        super().__init__(id=local_id, type=0, rank=rank) #uid
        self.iniCapital = capital

        self.labor=labor
        self.capital=capital
        self.capitalQ= 0
        self.unavailableLabor=0
        self.unavailableCapitalQ=0
        self.minOrderDuration=minOrderDuration
        self.maxOrderDuration=maxOrderDuration
        self.recipe = recipe
        self.laborProductivity=laborProductivity
        self.maxOrderProduction=maxOrderProduction
        self.assetsUsefulLife=assetsUsefulLife
        self.plannedMarkup=plannedMarkup
        self.orderObservationFrequency=orderObservationFrequency
        self.productionType=productionType
        self.sectorialClass=sectorialClass 
        
        self.lostProduction=0
        self.inventories=0
        self.inProgressInventories=0
        self.appRepository=[] #aPP is aProductiveProcess
        
        self.profits=0
        self.revenues=0
        self.totalCosts=0
        self.totalCostOfLabor=0
        self.totalCostOfCapital=0
        self.addedValue=0
        self.initialInventories=0
        self.grossInvestmentQ=0
        self.myBalancesheet=np.zeros((params['howManyCycles'], 20))

        self.movAvQuantitiesInEachPeriod=[]
        self.movAvDurations=[]
        
        self.productiveProcessIdGenerator=0
      
        self.theCentralPlanner=0

        self.Deposits = []
        self.Loans = []
        
        
        
        
    # activated by the Model
    def estimatingInitialPricePerProdUnit(self):

        total =  (1/self.laborProductivity)*params['wage']
        total += (1/self.laborProductivity)*self.recipe*params['costOfCapital']/params['timeFraction']
        total += (1/self.laborProductivity)*self.recipe/(self.assetsUsefulLife * params['timeFraction']) 
        if params['usingMarkup']: total *= (1+self.plannedMarkup)
        total *= ((self.maxOrderDuration+self.minOrderDuration)/2)
        return total       
        
        
    def settingCapitalQ(self, investmentGoodPrices):
        #############pt temporary solution
        #we temporary use this vector with a unique position as there is only one investment good at the moment
        self.priceOfDurableProductiveGoodsPerUnit = investmentGoodPrices[0] #1 
        self.currentPriceOfDurableProductiveGoodsPerUnit = investmentGoodPrices[0] #1  # the price to be paid to acquire 
                                                                                    # new capital in term of quantity
            
        #pt TMP TMP TMP TMP TMP TMP TMP TMP TMP TMP TMP TMP TMP TMP TMP TMP TMP TMP TMP
        #############   underlying idea:
        #               the actual initial price of durable productive goods (per unit of quantity) must be
        #               consistent with the initial cost of production of the durable productive goods;
        #
        #               the recipe set the ratio K/L where K is expressed in value;
        #
        #               having a price we implicitly set the "quantity";
        #
        #               substitution costs will consider both the change of the quantity and of the price
        #               at which the firm will pay the new productive goods;
        #
        #               the used v. unused capital measures are calculated as addenda of the capital in quantity
        #
        #               the costOfCapital (ratio of interests or rents) will be applied to the current value
        #               of the capital, after calculating the changes in quantity and then in value (considering 
        #               changes in q. and their value using the price of the new acquisitions)
        #
        #               as it evolves over time, the mean price of durable productive goods is an idiosyncratic
        #               property of the firm
        #
        #               L productivity is expressed in quantity as orders are expressed in quantity 

        self.capitalQ=self.capital/self.priceOfDurableProductiveGoodsPerUnit

    
        
    def dealingMovAvElements(self, freq, x, y):
        
        self.movAvQuantitiesInEachPeriod.append(x/y)
        if len(self.movAvQuantitiesInEachPeriod) > freq: self.movAvQuantitiesInEachPeriod.pop(0) 
            
        self.movAvDurations.append(y)
        if len(self.movAvDurations) > freq: self.movAvDurations.pop(0)

        
    def receivingNewOrder(self, productionOrder: float, orderDuration):

        #creates a statistics of the values of the received order
        self.dealingMovAvElements(self.orderObservationFrequency, productionOrder, orderDuration)
        
        #decision on accepting or refusing the new order
        productionOrderQuantityByPeriod=productionOrder/orderDuration
        requiredLabor=np.ceil(productionOrderQuantityByPeriod/self.laborProductivity)
        requiredCapitalQ=requiredLabor*self.recipe/self.priceOfDurableProductiveGoodsPerUnit
        #if self.uid[0]==29 and self.uid[2]==2:
        #    print("***1",t(),"new order q. per period", productionOrderQuantityByPeriod,\
        #          "req L", requiredLabor, "L", self.labor)
        
        #create a new aPP or skip the order
        if requiredLabor <= self.labor and requiredCapitalQ <= self.capitalQ: 
            self.productiveProcessIdGenerator += 1
            productiveProcessId=(self.uid[0],self.uid[1],self.uid[2],self.productiveProcessIdGenerator)
            aProductiveProcess = ProductiveProcess(productiveProcessId,productionOrderQuantityByPeriod, \
                                                   requiredLabor, requiredCapitalQ, orderDuration,\
                                                   self.priceOfDurableProductiveGoodsPerUnit,\
                                                   self.assetsUsefulLife)
            self.appRepository.append(aProductiveProcess)


    def produce(self, current_time)->tuple: 
        
        #total values of the firm in the current interval unit
        self.currentTotalCostOfProductionOrder=0
        self.currentTotalOutput=0
        self.currentTotalCostOfUnusedFactors=0
        self.currentTotalLostProduction=0
        self.currentTotalCostOfLostProduction=0
        
        avgRequiredLabor=0
        avgRequiredCapitalQ=0
        
        if current_time==0: self.initialInventories=0 
        else: self.initialInventories=self.inventories+self.inProgressInventories

        # activity within a time unit
        for aProductiveProcess in self.appRepository:  

            if not aProductiveProcess.hasResources and \
                        (self.labor - self.unavailableLabor >= aProductiveProcess.requiredLabor and\
                         self.capitalQ - self.unavailableCapitalQ >= aProductiveProcess.requiredCapitalQ):
                self.unavailableLabor += aProductiveProcess.requiredLabor
                self.unavailableCapitalQ += aProductiveProcess.requiredCapitalQ
                aProductiveProcess.hasResources = True 
                    
            if aProductiveProcess.hasResources: #resources may be just assigned above
                #production
                (aPPoutputOfThePeriod, aPPrequiredLabor, aPPrequiredCapitalQ, aPPlostProduction,\
                 aPPcostOfLostProduction) = aProductiveProcess.step()
                     
                self.currentTotalOutput += aPPoutputOfThePeriod
                
                cost = aPPrequiredLabor*params['wage'] \
                       + aPPrequiredCapitalQ*self.priceOfDurableProductiveGoodsPerUnit \
                                                         *params['costOfCapital']/params['timeFraction']\
                       + aPPrequiredCapitalQ*self.priceOfDurableProductiveGoodsPerUnit/ \
                         (self.assetsUsefulLife * params['timeFraction'])             
                                                       
                self.currentTotalCostOfProductionOrder += cost
                
                self.currentTotalLostProduction += aPPlostProduction
                self.currentTotalCostOfLostProduction += aPPcostOfLostProduction               
        
                if not params['usingMarkup']: self.plannedMarkup=0
                if aProductiveProcess.failure:
                    #consider markup
                    self.inProgressInventories -= cost*(aProductiveProcess.productionClock-1)*(1+self.plannedMarkup)
                    
                    #NB this is an approximation because in multiperiodal production processes the
                    #   priceOfDurableProductiveGoodsPerUnit may change, but it is a realistic
                    #   approximation in firm accounting               
                    
                else:
                    if aProductiveProcess.productionClock < aProductiveProcess.orderDuration:
                        self.inProgressInventories += cost * (1+self.plannedMarkup) #consider markup
                    else:
                        self.inventories+=cost*aProductiveProcess.orderDuration*(1+self.plannedMarkup)
                        self.inProgressInventories -= cost*(aProductiveProcess.orderDuration-1) *(1+self.plannedMarkup)
                        #consider markup (it is added in the final and subtracted by the inProgress)

        self.currentTotalCostOfUnusedFactors =  (self.labor - self.unavailableLabor)*params['wage'] + \
                                        (self.capitalQ - self.unavailableCapitalQ)*\
                                         self.priceOfDurableProductiveGoodsPerUnit*\
                                         params['costOfCapital']/params['timeFraction'] + \
                                         (self.capitalQ - self.unavailableCapitalQ) *\
                                            self.priceOfDurableProductiveGoodsPerUnit/ \
                                            (self.assetsUsefulLife * params['timeFraction'])
                                         # considering substitutions also for the idle capital
        
        #print("ORDER MOV AV",self.uid, sum(self.movAvQuantitiesInEachPeriod)/ len(self.movAvQuantitiesInEachPeriod), flush=True)
        avgRequiredLabor=np.ceil( ((sum(self.movAvQuantitiesInEachPeriod)/len(self.movAvQuantitiesInEachPeriod)) /self.laborProductivity )\
                *( sum(self.movAvDurations)/ len(self.movAvDurations) ))
        
        #total cost of labor
        self.totalCostOfLabor= self.labor*params['wage']
        
        #labor adjustments (frequency at orderObservationFrequency)
        if current_time % self.orderObservationFrequency == 0 and current_time > 0:
            if self.labor > (1+params['tollerance']) * avgRequiredLabor:
                self.labor = np.ceil((1+params['tollerance']) * avgRequiredLabor) #max accepted q. of L (firing)
            if self.labor < (1/(1+params['tollerance'])) * avgRequiredLabor:
                self.labor = np.ceil((1/(1+params['tollerance'])) * avgRequiredLabor) #min accepted q. of L (hiring)
            #if self.uid==(32,0,0): print("***",self.uid, "labM",avgRequiredLabor,"L", self.labor, flush=True)
           
        
        #capital adjustments (frequency at each cycle)
        #here the following variables are disambiguated between actual and desired values, so they appear in a double shape:
        # i) capital and capitalQ, ii) desiredCapitalSubstistutions and desiredCapitalQsubstitutions
        
        self.capitalBeforeAdjustment=self.capital
        desiredCapitalQsubstitutions=0
        desiredCapitalSubstitutions=0
        requiredCapitalQincrement=0
        requiredCapitalIncrement=0
        
        if current_time > self.orderObservationFrequency: #no corrections before the end of the first correction interval
                                                 #where orders are under the standard flow of the firm
            capitalQmin= self.capitalQ/(1+params['tollerance'])
            capitalQmax= self.capitalQ*(1+params['tollerance'])
            
            avgRequiredCapital=avgRequiredLabor*self.recipe
            avgRequiredCapitalQ=avgRequiredCapital/self.currentPriceOfDurableProductiveGoodsPerUnit
            
            requiredCapitalSubstitution=self.capital/(self.assetsUsefulLife * params['timeFraction'])
            requiredCapitalSubstitutionQ=self.capitalQ/(self.assetsUsefulLife * params['timeFraction']) 
            
            #obsolescence  and deterioration effect
            self.capitalQ-=requiredCapitalSubstitutionQ
            self.capital-=requiredCapitalSubstitution
            
            a=(-requiredCapitalSubstitutionQ)
            #A=(-requiredCapitalSubstitution)
            
            #case I
            if avgRequiredCapitalQ < capitalQmin:
                b=avgRequiredCapitalQ-capitalQmin #being b<0
                #quantities
                if b<=a: desiredCapitalQsubstitutions=0
                if b>a: desiredCapitalQsubstitutions=abs(a)-abs(b)

                #values
                desiredCapitalSubstitutions=desiredCapitalQsubstitutions*self.currentPriceOfDurableProductiveGoodsPerUnit
            
            #case II
            if capitalQmin <= avgRequiredCapitalQ and avgRequiredCapitalQ <= capitalQmax:
                #quantities
                desiredCapitalQsubstitutions=abs(a) 
    
                #values
                desiredCapitalSubstitutions=desiredCapitalQsubstitutions*self.currentPriceOfDurableProductiveGoodsPerUnit
            
            #case III
            if avgRequiredCapitalQ > capitalQmax:
                #quantities
                desiredCapitalQsubstitutions=abs(a)
                requiredCapitalQincrement=avgRequiredCapitalQ-capitalQmax

                #values
                desiredCapitalSubstitutions=desiredCapitalQsubstitutions*self.currentPriceOfDurableProductiveGoodsPerUnit
                requiredCapitalIncrement=requiredCapitalQincrement*self.currentPriceOfDurableProductiveGoodsPerUnit
        
        
        self.desiredCapitalQsubstitutions=desiredCapitalQsubstitutions
        self.requiredCapitalQincrement=requiredCapitalQincrement                
        self.desiredCapitalSubstitutions=desiredCapitalSubstitutions
        self.requiredCapitalIncrement=requiredCapitalIncrement

        
        #=========================================================================================
            # the key local variables are:
            #
            # desiredCapitalQsubstitutions & desiredCapitalSubstitutions
            #
            # requiredCapitalQincrement & requiredCapitalIncrement
            #
            #
            # giving:
            #
            # self.capitalQ increment (with +=) from desiredCapitalQsubstitutions + requiredCapitalQincrement
            #
            # self.capital increment (with +=) from desiredCapitalSubstitutions + requiredCapitalIncrement
            #
            # self.grossInvestmentQ = desiredCapitalQsubstitutions+requiredCapitalQincrement, for reporting reasons
            #
            #
            # with return:
            #
            # self.capital
            # self.grossInvestmentQ
            #=========================================================================================

    def allowInformationToCentralPlanner(self) -> tuple:
        return(self.desiredCapitalQsubstitutions, self.requiredCapitalQincrement,\
               self.desiredCapitalSubstitutions, self.requiredCapitalIncrement)
    
    
    def requestGoodsToTheCentralPlanner(self) -> tuple:
        return(self.desiredCapitalQsubstitutions,self.requiredCapitalQincrement,\
                           self.desiredCapitalSubstitutions, self.requiredCapitalIncrement)
    
    
    def concludeProduction(self):
        
        #action of the planner
        capitalQsubstitutions = self.investmentGoodsGivenByThePlanner[0]
        capitalQincrement = self.investmentGoodsGivenByThePlanner[1]
        capitalSubstitutions = self.investmentGoodsGivenByThePlanner[2]
        capitalIncrement = self.investmentGoodsGivenByThePlanner[3]
        
        
        #effects
        self.capitalQ+=capitalQsubstitutions+capitalQincrement 
        self.capital+=capitalSubstitutions+capitalIncrement
        self.grossInvestmentQ=capitalQsubstitutions+capitalQincrement

        
        
        #total cost of capital
        self.totalCostOfCapital=self.capitalBeforeAdjustment*params['costOfCapital']/params['timeFraction']\
                                +capitalQsubstitutions*self.currentPriceOfDurableProductiveGoodsPerUnit
           

        # remove concluded aPPs from the list (backward to avoid skipping when deleting)
        for i in range(len(self.appRepository)-1,-1,-1):
            if self.appRepository[i].productionClock == self.appRepository[i].orderDuration: 
                self.unavailableLabor-=self.appRepository[i].requiredLabor
                self.unavailableCapitalQ-=self.appRepository[i].requiredCapitalQ
                del self.appRepository[i]

        return(self.currentTotalOutput, self.currentTotalCostOfProductionOrder, self.currentTotalCostOfUnusedFactors,self.inventories,\
               self.inProgressInventories, self.currentTotalLostProduction, self.currentTotalCostOfLostProduction, \
               self.labor, self.capital, self.grossInvestmentQ)
               # labor, capital modified just above
        

    def receiveSellingOrders(self, shareOfInventoriesBeingSold: float, centralPlannerBuyingPriceCoefficient: float):
        nominalQuantitySold=shareOfInventoriesBeingSold*self.inventories
        self.revenues=centralPlannerBuyingPriceCoefficient*nominalQuantitySold
        self.inventories-=nominalQuantitySold    
         
    def makeBalancesheet(self, current_time):
        self.totalCosts= self.currentTotalCostOfProductionOrder + self.currentTotalCostOfUnusedFactors
        """
        if params['usingMarkup']:
            self.inventories *= (1+self.plannedMarkup) #planned because != ex post
            self.inProgressInventories *= (1+self.plannedMarkup) 
        """
        
        self.profits= self.revenues+(self.inventories + self.inProgressInventories)\
                    -self.totalCosts-self.initialInventories 
        self.addedValue=self.profits+self.totalCosts
        
        self.myBalancesheet[current_time, 0]=self.sectorialClass #i.e. row number in firms-features
        
        self.myBalancesheet[current_time, 1]=self.initialInventories
        self.myBalancesheet[current_time, 2]=self.totalCosts
        
        if not self.productionType in params["investmentGoods"]: self.myBalancesheet[current_time, 3]=self.revenues
        else: self.myBalancesheet[current_time, 4]=self.revenues

        if not self.productionType in params["investmentGoods"]: self.myBalancesheet[current_time, 5]=self.inventories
        else: self.myBalancesheet[current_time, 6]=self.inventories 
            
        if not self.productionType in params["investmentGoods"]: self.myBalancesheet[current_time, 7]=self.inProgressInventories
        else: self.myBalancesheet[current_time, 8]=self.inProgressInventories
        
        self.myBalancesheet[current_time, 9]=self.profits
        self.myBalancesheet[current_time, 10]=self.addedValue
        self.myBalancesheet[current_time, 11]=self.currentTotalOutput
        self.myBalancesheet[current_time, 12]=self.currentTotalCostOfProductionOrder
        self.myBalancesheet[current_time, 13]=self.currentTotalCostOfUnusedFactors
        self.myBalancesheet[current_time, 14]=self.currentTotalLostProduction
        self.myBalancesheet[current_time, 15]=self.currentTotalCostOfLostProduction
        self.myBalancesheet[current_time, 16]=self.totalCostOfLabor
        self.myBalancesheet[current_time, 17]=self.totalCostOfCapital
        self.myBalancesheet[current_time, 18]=self.grossInvestmentQ
        self.myBalancesheet[current_time, 19]=self.productionType
        
        
    
    def save(self) -> Tuple: # mandatory, used by request_agents and by synchroniza
        """
        Saves the state of the Firm as a Tuple.

        Returns:
            The saved state of this instance of Firm.
        """
        # ??the structure of the save is ( ,( )) due to an incosistent use of the 
        # save output in update internal structure /fixed in v. 1.1.2???)
        return (self.uid,(self.labor,self.capital,self.minOrderDuration,self.maxOrderDuration,self.recipe,\
                self.laborProductivity,self.maxOrderProduction,self.assetsUsefulLife,self.plannedMarkup,\
                self.orderObservationFrequency,self.productionType,self.sectorialClass))

    def update(self, dynState: Tuple): # mandatory, used by synchronize
        self.labor = dynState[0]
        self.capital = dynState[1]
        self.minOrderDuration = dynState[2]
        self.maxOrderDuration = dynState[3]
        self.recipe = dynState[4]
        self.laborProductivity = dynState[5]
        self.maxOrderProduction = dynState[6]
        self.assetsUsefulLife = dynState[7]
        self.plannedMarkup = dynState[8]
        self.orderObservationFrequency = dynState[9]
        self.productionType = dynState[10]
        self.sectorialClass = dynState[11]

############################################################################################################################
###########################################################################################################################


class ProductiveProcess():
    def __init__(self, productiveProcessId: tuple, targetProductionOfThePeriod:float, requiredLabor:int,\
                 requiredCapitalQ:float, orderDuration:int, priceOfDurableProductiveGoodsPerUnit:float,\
                 assetsUsefulLife:float):
        
        self.targetProductionOfThePeriod=targetProductionOfThePeriod
        self.requiredLabor = requiredLabor
        self.requiredCapitalQ = requiredCapitalQ
        self.orderDuration = orderDuration
        self.productionClock=0
        self.hasResources= False
        self.productiveProcessId=productiveProcessId
        self.priceOfDurableProductiveGoodsPerUnit=priceOfDurableProductiveGoodsPerUnit
        self.assetsUsefulLife=assetsUsefulLife
        
    #def step(self, productionOrder)->tuple:
    def step(self)->tuple:
        
        lostProduction=0
        costOfLostProduction=0
        self.productionClock += 1
        self.failure=False
        
        # production failure
        if params['probabilityToFailProductionChoices'] >= rng.random():
            self.failure=True
            #if self.productiveProcessId[0]==29 and self.productiveProcessId[2]==2:
            #    print("***2",t(),"failure")
            #print("failure",flush=True)
            lostProduction=self.targetProductionOfThePeriod*self.productionClock
            self.targetProductionOfThePeriod=0
            costOfLostProduction=(params['wage']* self.requiredLabor+\
                                       (params['costOfCapital']/params['timeFraction'])* self.requiredCapitalQ*\
                                        self.priceOfDurableProductiveGoodsPerUnit)*self.productionClock+\
                                        (self.requiredCapitalQ*self.priceOfDurableProductiveGoodsPerUnit)/ \
                                        (self.assetsUsefulLife * params['timeFraction']) 
            self.orderDuration = self.productionClock   

        return(self.targetProductionOfThePeriod, self.requiredLabor, self.requiredCapitalQ, \
               lostProduction, costOfLostProduction)

    

############################################################################################################################
############################################################################################################################



class CentralPlannerReporter(core.Agent):
    
    def __init__(self, local_id: 0, rank: 0):
        super().__init__(id=local_id, type=params['CEN_PLANNER_REPORTER_TYPE'], rank=rank) #uid
    
        self.informationTableLastCol=[0,0,0,0] # superflous
        self.invGoodsDemandList=[0,0,0,0] #to avoid an error in first sync

    def informationTableLastCols(self,c0,c1,c2,c3):
        
        #workingMultiRank
        self.informationTableLastCol=[]
        self.informationTableLastCol.append(c0)
        self.informationTableLastCol.append(c1)
        self.informationTableLastCol.append(c2)
        self.informationTableLastCol.append(c3)

    def invGoodsDemand(self,d0,d1,d2,d3):
        #workingMultiRank
        self.invGoodsDemandList=[]
        self.invGoodsDemandList.append(d0)
        self.invGoodsDemandList.append(d1)
        self.invGoodsDemandList.append(d2)
        self.invGoodsDemandList.append(d3)
        #print("from invGoodsDemand, rank=",rank,"t=",t(),self.invGoodsDemandList,flush=True)
    
    def save(self) -> Tuple: # mandatory, used by request_agents and by synchronizazion
        """
        Saves the state of the CentralPlannerReporter as a Tuple.

        Returns:
            The saved state of this CentralPlannerReporter.
        """
        # ??the structure of the save is ( ,( )) due to an incosistent use of the 
        # save output in update internal structure /fixed in v. 1.1.2???)
        #return (self.uid,(self.informationTable,)) #the comma is relevant for positional reasons
        #print(rank, "save",self.informationTableLastCol,flush=True)
        return (self.uid,(self.informationTableLastCol,self.invGoodsDemandList,))

    def update(self, dynState: Tuple): # mandatory, used by synchronize
        #print(rank, "updt",dynState,flush=True)
        #print("from reporter upddat, rank=",rank,"t=",t(),dynState,flush=True)
        for i in range(4):
            self.informationTableLastCol[i]=dynState[0][i]
        for i in range(4):
            self.invGoodsDemandList[i]=dynState[1][i]

# class CentralBankReporter(core.Agent):
#     
#     def __init__(self, local_id=0, rank=0):
#         super().__init__(id=local_id, type=params['CEN_BANK_REPORTER_TYPE'], rank=rank) #uid
# 
#         # changeReverse, totalReverse
#         self.informationTableLastCol=[0,0] # superflous
#         self.connectCentralBank = None
# 
# 
#     def reciveInformationLastCol(self):
#         self.informationTableLastCol= self.connectBank.getInformationTableData()
#     
#     def save(self) -> Tuple: # mandatory, used by request_agents and by synchronizazion
#         """
#         Saves the state of the CentralPlannerReporter as a Tuple.
# 
#         Returns:
#             The saved state of this CentralPlannerReporter.
#         """
#         # ??the structure of the save is ( ,( )) due to an incosistent use of the 
#         # save output in update internal structure /fixed in v. 1.1.2???)
#         #return (self.uid,(self.informationTable,)) #the comma is relevant for positional reasons
#         #print(rank, "save",self.informationTableLastCol,flush=True)
#         return (self.uid,(self.informationTableLastCol,))
# 
#     def update(self, dynState: Tuple): # mandatory, used by synchronize
#         #print(rank, "updt",dynState,flush=True)
#         #print("from reporter upddat, rank=",rank,"t=",t(),dynState,flush=True)
#         self.informationTableLastCol=dynState[0]



In [ ]:
agent_cache = {}

def restore_agent(agent_data: Tuple):

    uid=agent_data[0]
    
    # adaptive old version
    if len(agent_data) > 1 and len(agent_data[1]) >= 4:
        isGlobal, paramGroup = agent_data[1][2], agent_data[1][3]
 
    if uid[1] == params['FIRM_TYPE']:
    
        if uid in agent_cache: 
            tmp = agent_cache[uid] # found
            tmp.labor = agent_data[1][0] #restore data
            tmp.capital = agent_data[1][1]
            tmp.minOrderDuration = agent_data[1][2]
            tmp.maxOrderDuration = agent_data[1][3]
            tmp.recipe = agent_data[1][4]
            tmp.laborProductivity = agent_data[1][5]
            tmp.maxOrderProduction = agent_data[1][6]
            tmp.assetsUsefulLife = agent_data[1][7]
            tmp.plannedMarkup = agent_data[1][8]
            tmp.orderObservationFrequency = agent_data[1][9]
            tmp.productionType = agent_data[1][10]
            tmp.sectorialClass = agent_data[1][11]
            
        else: #creation of an instance of the class with its data
            tmp = Firm(uid[0], uid[2],agent_data[1][0],agent_data[1][1],agent_data[1][2],agent_data[1][3],\
                       agent_data[1][4],agent_data[1][5],agent_data[1][6],agent_data[1][7],agent_data[1][8],\
                       agent_data[1][9],agent_data[1][10],agent_data[1][11])
            agent_cache[uid] = tmp
            
        return tmp

    if uid[1] == params['CEN_PLANNER_TYPE']:
    
        if uid in agent_cache: 
            raise ValueError('should not happen')
            tmp = agent_cache[uid] # found
            
        else: #creation of an instance of the class with its data
            tmp = CentralPlanner(uid, params, isGlobal, paramGroup, **(params['agent_params'][uid[1]][paramGroup] or {}))
            agent_cache[uid] = tmp
            
        return tmp
    
    if uid[1] == params['CEN_PLANNER_TYPE'] + 1000:
    
        if uid in agent_cache: 
            tmp = agent_cache[uid] # found
            
        else: #creation of an instance of the class with its data
            tmp = AgentReporter(uid[0], uid[1], uid[2])
            agent_cache[uid] = tmp
            #tmp.incrementAndSubstitutions = agent_data[1][0] #not used, variable defined in init
            
        return tmp


    if uid[1] == params['CEN_PLANNER_REPORTER_TYPE']:
    
        if uid in agent_cache: 
            tmp = agent_cache[uid] # found
            #tmp.informationTable = agent_data[1][0] #restore data
            
        else: #creation of an instance of the class with its data
            tmp = CentralPlannerReporter(uid[0], uid[2])
            agent_cache[uid] = tmp
            
        return tmp


    if uid[1] == params['COMM_BANK_TYPE']:
    
        if uid in agent_cache: 
            raise ValueError('CommercialBank should only create or update')
            tmp = agent_cache[uid] # found
            #tmp.informationTable = agent_data[1][0] #restore data
            
        else: #creation of an instance of the class with its data
            tmp = CommercialBank(uid, params, isGlobal, paramGroup, **(params['agent_params'][uid[1]][paramGroup] or {}))
            agent_cache[uid] = tmp
            
        return tmp

    if uid[1] == (params['COMM_BANK_TYPE'] + 1000):
    
        if uid in agent_cache: 
            raise ValueError('reporter should only create or update')
            tmp = agent_cache[uid] # found
            #tmp.informationTable = agent_data[1][0] #restore data
            
        else: #creation of an instance of the class with its data
            
            tmp = AgentReporter(uid[0], uid[1], uid[2])
            agent_cache[uid] = tmp
            
        return tmp
    
    if uid[1] == params['CEN_BANK_TYPE']:
    
        if uid in agent_cache: 
            raise ValueError('CEN_BANK should only create or update')
            tmp = agent_cache[uid] # found
            #tmp.informationTable = agent_data[1][0] #restore data
            
        else: #creation of an instance of the class with its data
            
            tmp = CentralBank(uid, params, isGlobal, paramGroup, **(params['agent_params'][uid[1]][paramGroup] or {}))
            agent_cache[uid] = tmp
            
        return tmp
    
    if uid[1] == params['CEN_BANK_TYPE'] + 1000:
    
        if uid in agent_cache: 
            raise ValueError('CEN_BANK_REPORTER should only create or update')
            tmp = agent_cache[uid] # found
            #tmp.informationTable = agent_data[1][0] #restore data
            
        else: #creation of an instance of the class with its data
            
            tmp = AgentReporter(uid[0], uid[1], uid[2])
            agent_cache[uid] = tmp
        
        return tmp
            
            
    if uid[1] == params['CREDIT_MARKET_TYPE']:
    
        if uid in agent_cache: 
            raise ValueError('should not happen')
            tmp = agent_cache[uid] # found
            
        else: #creation of an instance of the class with its data
            tmp = CreditMarket(uid, params, isGlobal, paramGroup, **(params['agent_params'][uid[1]][paramGroup] or {}))
            agent_cache[uid] = tmp
            
        return tmp
    
    if uid[1] == params['CREDIT_MARKET_TYPE'] + 1000:
    
        if uid in agent_cache: 
            tmp = agent_cache[uid] # found
            
        else: #creation of an instance of the class with its data
            tmp = MarketReporter(uid[0], uid[1], uid[2])
            agent_cache[uid] = tmp
            #tmp.incrementAndSubstitutions = agent_data[1][0] #not used, variable defined in init
            
        return tmp
            

In [1]:
class Model:
    
    def __init__(self, comm, params: Dict):
        PARAMS = params
        self.rank    = comm.Get_rank()
        self.rankNum = comm.Get_size()
        self.rng = rng

        self.runner = schedule.init_schedule_runner(comm)
        self.context = ctx.SharedContext(comm)


        self.totalProduction=[]
        self.totalCostOfProduction=[]
        self.totalCostOfUnusedFactors=[]
        self.totalInvGoodsRevenues=[]
        self.totalConsGoodsRevenues=[]
        self.totalInvGoodsInventories=[]
        self.totalConsGoodsInventories=[]
        self.totalInProgressInvGoodsInventories=[]
        self.totalInProgressConsGoodsInventories=[]
        self.totalLostProduction=[]
        self.totalCostOfLostProduction=[]
        self.updatedLabor=[]
        self.updatedCapital=[]
        self.totalGrossInvestmentQ=[]
        self.firmData={}
        self.theCentralPlanner=0
        self.theCentralPlannerReporter=0
        self.theCentralPlannerReporterGhostList=[]
        
        self.theCreditMarket = None

        self.bankData={}
        self.bankStockData = {}
        self.agent_cache = {}
        
        self.global_agent_uid = GlobalContainer(params)

        #the context and the runner are created in step 1 
      
        self.runner.schedule_event(          0.0,     self.initGhosts) 
        self.runner.schedule_event(          0.0,     self.initInvestmentGoodPrices) 
        
        self.runner.schedule_repeating_event(0.0,  1, self.counter)
        

        self.runner.schedule_repeating_event(0.03, 1, self.bankPayLoan)
        self.runner.schedule_repeating_event(0.04, 1, self.bankConcludingInformation)
        self.runner.schedule_repeating_event(0.05, 1, self.bankGetCredit)
        self.runner.schedule_repeating_event(0.06, 1, self.bankConcludingInformation)
        

        self.runner.schedule_repeating_event(0.21, 1, self.plannerPreparingActions)
        self.runner.schedule_repeating_event(0.22, 1, self.plannerDiffusingProductionOrders)
        self.runner.schedule_repeating_event(0.23, 1, self.firmsProducing)
        self.runner.schedule_repeating_event(0.24, 1, self.plannerPreparingAndMakingDistributionOfInvGoods)
        self.runner.schedule_repeating_event(0.25,  1, self.firmsConcludingProduction)
        self.runner.schedule_repeating_event(0.26, 1, self.firmsMakingFinancialTransactionsRelatedToCosts)
        
        self.runner.schedule_repeating_event(0.4,  1, self.plannerGeneratingDemandOrders) #invGoods for next period investments
        self.runner.schedule_repeating_event(0.41, 1, self.firmsMakingFinancialTransactionsRelatedToRevenues)
        
        self.runner.schedule_repeating_event(0.51, 1, self.creditMarket)
        
        self.runner.schedule_repeating_event(0.7, 1, self.bankConcludingInformation)
        self.runner.schedule_repeating_event(0.71,  1, self.enterprisesMakingBalancesheet) #enterprises=firms+banks
        
        self.runner.schedule_stop(params['howManyCycles'])
        self.runner.schedule_end_event(self.finish)
             
        ####################################################################################################
        ###################################### CREATE AGENTS ##########################################
        ####################################################################################################
        
        ####################################################################################################
        ################################## CREATE CENTRAL PLANNER AGENT ####################################
        ####################################################################################################
        for agentUid, (paramGroup, isGlobal) in self.global_agent_uid.getRealAgents(agent_type=params['CEN_PLANNER_TYPE'], rankId=self.rank, uid_only=False):
            print((params['agent_params'][agentUid[1]][paramGroup] or {}))
            agent = CentralPlanner(agentUid, params=params, isGlobal=isGlobal, paramGroup=paramGroup,  **(params['agent_params'][agentUid[1]][paramGroup] or {}))
            self.context.add(agent)

        
        self.iniGhostAgent(params['CEN_PLANNER_TYPE'], AgentReporter)
        
        

        self.iniAgents()
        
        for agentUid, (paramGroup, isGlobal) in self.global_agent_uid.getRealAgents(agent_type=params['CREDIT_MARKET_TYPE'], rankId=self.rank, uid_only=False):
            print((params['agent_params'][agentUid[1]][paramGroup] or {}))
            agent = CreditMarket(agentUid, params=params, isGlobal=isGlobal, paramGroup=paramGroup,  **(params['agent_params'][agentUid[1]][paramGroup] or {}))
            self.context.add(agent)

        
        self.iniGhostAgent(params['CREDIT_MARKET_TYPE'], MarketReporter)

            
        ####################################################################################################
        ###################################### CREATE FIRM AGENTS ##########################################
        ####################################################################################################
        
        #importing csv file containing info about firms 
        #share of firms of that class, L min, L max, K min, K max, order duration min, order duration max, 
        #recipe, L prod, max order production, assets' useful life, planned markup, 
        #order observation frequency min, order observation frequency max, production type
        with open('firm-features.csv', newline='') as csvfile:
            firmReader= csv.reader(csvfile, delimiter=',', quoting=csv.QUOTE_NONNUMERIC)
            
            self.rowNumber=-1
            k=0
            #for each record in firm-features.csv
            #share of firms of that class, L min, L max, K min, K max, order duration min, order duration max, recipe, 
            #L prod, max order production, assets' useful life, planned markup, order observation frequency min, 
            #order observation frequency max, production type
            for row in firmReader:
                print(row)
                if self.rowNumber>=0:
                    for i in range(int(row[0] * params['Firm.count'])// rankNum):
                        labor= rng.integers(row[1], row[2]+1) #+1 because integers exclude extremes
                        capital= row[3] + rng.random()*(row[4] -row[3])
                        minOrderDuration= row[5]
                        maxOrderDuration= row[6]
                        recipe= row[7] #K/L 
                        laborProductivity= row[8]
                        maxOrderProduction= row[9]
                        avgAssetsUsefulLife=row[10]  #https://www.oecd.org/sdd/productivity-stats/43734711.pdf
                        plannedMarkup=row[11]
                        orderObservationFrequency=rng.integers(row[12], row[13]+1)
                        productionType=int(row[14]) #productionType in firm-features.csv indicates the production of
                                                #investment goods if it is into the investmentGoods list in yaml
                        sectorialClass=int(self.rowNumber)
                        aFirm =Firm(k, self.rank, labor, capital, minOrderDuration, maxOrderDuration, recipe, laborProductivity,\
                                maxOrderProduction, avgAssetsUsefulLife, plannedMarkup, orderObservationFrequency, productionType,\
                                sectorialClass)

                        self.context.add(aFirm)
                        k += 1 # first element of the UID of the agents
                self.rowNumber += 1
            self.firmCount=k #one more, here is a count, not an id
        
        
        if self.rank == 0:
            cb = self.context.agent((0, params['CEN_BANK_TYPE'], 0))
        else:
            cb = self.context.ghost_agent((0, params['CEN_BANK_TYPE'], 0))
        
        self.theCentralBank = cb
    
        # Initialize reserves for real and ghost banks
        for uid, (paramGroup, isGlobal) in self.global_agent_uid.getRealAgents(agent_type=params['COMM_BANK_TYPE'], uid_only=False):
            if uid[2] == self.rank:
                aBank = self.context.agent(uid)
                reserve = Reserve(1000, aBank, cb)
                aBank.Reserves.append(reserve)
                cb.Reserves.append(reserve)
            elif uid[2] != self.rank and isGlobal:
                aBank = self.context.ghost_agent(uid)
                # ini with 0
                reserve = Reserve(0, aBank, cb)
                aBank.Reserves.append(reserve)
                cb.Reserves.append(reserve)
                
        
        # Initialize reserves for real and ghost CentralPlanner
        if self.rank == 0:
            cp = self.context.agent((0, params['CEN_PLANNER_TYPE'], 0))
        else:
            cp = self.context.ghost_agent((0, params['CEN_PLANNER_TYPE'], 0))
        
        self.theCentralPlanner = cp
        
        fist_global_bank = None
        for uid, (paramGroup, isGlobal) in self.global_agent_uid.getRealAgents(agent_type=params['COMM_BANK_TYPE'], uid_only=False):
            if isGlobal:
                fist_global_bank = uid
        
        if fist_global_bank[2] == self.rank:
            abank = self.context.agent(fist_global_bank)
        else:
            abank = self.context.ghost_agent(fist_global_bank)
            
        deposit = Deposit(10000, cp, abank)
        cp.Deposits.append(deposit)
        abank.Deposits.append(deposit)
        
        # Initialize reserves for real and ghost Firm
        for aFirm in self.context.agents(params['FIRM_TYPE']):
            bank_uid  = self.global_agent_uid.getRandomAllAgent(agent_type=params['COMM_BANK_TYPE'], rankId= self.rank)

            if bank_uid[2] == self.rank:
                abank = self.context.agent(bank_uid)
            else:
                abank = self.context.ghost_agent(bank_uid)

            deposit = Deposit(aFirm.capital, aFirm, abank)
            aFirm.Deposits.append(deposit)
            abank.Deposits.append(deposit)
            
            aFirm.theCentralPlanner = self.theCentralPlanner
            
        if self.rank == 0:
            self.theCreditMarket = self.context.agent((0, params['CREDIT_MARKET_TYPE'], 0))
        else:
            self.theCreditMarket = self.context.ghost_agent((0, params['CREDIT_MARKET_TYPE'], 0))
            

    
    def t(self):
        return int(self.runner.schedule.tick)
    
    def initGhosts(self):
        pass
    
    def iniGhostAgent(self, agentType, reporterClass):

        # call bank ghost if it is a global agent
        ghostToRequest = []

        for agentUid, (sectorialClass, isGlobal) in self.global_agent_uid.getRealAgents(agent_type=agentType, uid_only=False):
            if agentUid[2] != self.rank and isGlobal:
                ghostToRequest.append((agentUid, agentUid[2]))

        self.context.request_agents(ghostToRequest,restore_agent)

        
        # call reporter
        for agentUid, (sectorialClass, isGlobal) in self.global_agent_uid.getRealAgents(agent_type=agentType, uid_only=False):
            if agentUid[2] != self.rank and isGlobal:
                agent = self.context.ghost_agent(agentUid)
                
                uniqe_id = (agentUid[0] + agentUid[2]) * (agentUid[0] + agentUid[2] + 1) // 2 + agentUid[2]
                angentReporter = reporterClass(uniqe_id, agentType+1000, self.rank) #local_id=0, rank=rank
                angentReporter.connectAgent = agent
                self.context.add(angentReporter)

        # call reporter ghost
        reporterToRequest = []

        for agentUid, (sectorialClass, isGlobal) in self.global_agent_uid.getRealAgents(agent_type=agentType, uid_only=False):
            if agentUid[2] == self.rank and isGlobal:
                for i in range(rankNum):
                    if i != self.rank:
                        uniqe_id = (agentUid[0] + agentUid[2]) * (agentUid[0] + agentUid[2] + 1) // 2 + agentUid[2]
                        reporterToRequest.append(((uniqe_id, agentType+1000, i), i))
        
        self.context.request_agents(reporterToRequest,restore_agent)

        # connect the main agent with reporter ghost
        for agentUid, (sectorialClass, isGlobal) in self.global_agent_uid.getRealAgents(agent_type=agentType, uid_only=False):
            if agentUid[2] == self.rank and isGlobal:
                agent = self.context.agent(agentUid)
                for i in range(rankNum):
                    if i != self.rank:
                        uniqe_id = (agentUid[0] + agentUid[2]) * (agentUid[0] + agentUid[2] + 1) // 2 + agentUid[2]
                        agent.reporterGhostList.append(self.context.ghost_agent((uniqe_id, agentType+1000, i)))

    def iniAgents(self):
        
        # create local bank agents
        for agentUid, (paramGroup, isGlobal) in self.global_agent_uid.getRealAgents(agent_type=params['COMM_BANK_TYPE'], rankId=self.rank, uid_only=False):

            agent = CommercialBank(agentUid, params=params, isGlobal=isGlobal, paramGroup=paramGroup,  **params['agent_params'][agentUid[1]][paramGroup])
            self.context.add(agent)

        # create local central bank agents
        for agentUid, (paramGroup, isGlobal) in self.global_agent_uid.getRealAgents(agent_type=params['CEN_BANK_TYPE'], rankId=self.rank, uid_only=False):

            agent = CentralBank(agentUid, params=params, isGlobal=isGlobal, paramGroup=paramGroup,  **(params['agent_params'][agentUid[1]][paramGroup] or {}))
            self.context.add(agent)


        self.iniGhostAgent(params['COMM_BANK_TYPE'], AgentReporter)
        self.iniGhostAgent(params['CEN_BANK_TYPE'], AgentReporter)


    
    #initialize investment good prices
    def initInvestmentGoodPrices(self):
        self.investmentGoodPrices=[0]*len(params['investmentGoods'])
        
        for anInvGoodType in range(len(params['investmentGoods'])):
            count=0
            for aFirm in self.context.agents(agent_type=params['FIRM_TYPE']):
                if aFirm.productionType == params['investmentGoods'][anInvGoodType]:
                    self.investmentGoodPrices[anInvGoodType]+=aFirm.estimatingInitialPricePerProdUnit()
                    count+=1
            if count != 0: self.investmentGoodPrices[anInvGoodType]/=count
        
        if not any(self.investmentGoodPrices): 
            print("\nThere are no investment goods!")
            sys.exit(0)
        
        for aFirm in self.context.agents(agent_type=params['FIRM_TYPE']):
            aFirm.settingCapitalQ(self.investmentGoodPrices)
            if aFirm.uid[0]==0: print("rank",self.rank,"Initial price of durable productive goods per unit",\
                                      aFirm.priceOfDurableProductiveGoodsPerUnit, flush=True) #as an info to the user
                

    #count the cycles number
    def counter(self):
        if int(self.t()) % params["tickNumber.betweenChecks"] == 0 and self.t()>9: 
            print("rank", self.rank, "tick", self.t(), \
                  "proportionalValue",self.theCentralPlanner.proportionalValue, flush=True)
            

    def plannerPreparingActions(self): 
        #workingMultiRank, rules are the same
        self.theCentralPlanner.preparingActions(self) # self here is the model instance
        # step made in paraller independly in all the ranks using the infos of plannerMethods.py
        # 
        ##
        ##sinchronize ghosts
        ##
         #theCentralPlanner diffuse infos to its ghosts
                                                           #from rank 0 to the other ranks (currentry nothing
                                                           #interesting)
                                                           #theCentralPlannerReporter send infos to its ghost
                                                           # from rank !=0 to rank 0
        if self.rankNum > 1 and self.context.contains_type(agent_type=params['CEN_PLANNER_TYPE'] + 1000):
            for aReporter in self.context.agents(agent_type=params['CEN_PLANNER_TYPE'] + 1000):
                aReporter.reciveInformationLastCol()

        
        if self.rankNum > 1: self.context.synchronize(restore_agent)
        
        #test
        #if rank==0:
        #    for i in range(1,rankNum):
        #        print(rank, t(), self.theCentralPlannerReporterGhostList[i-1].informationTableLastCol,flush=True)

        #add data collected from central planner reporter of ranks > 0 to the central planner of rank 0 data
        
        # 
        self.theCentralPlanner.mergeInformationTableData()
        
        
        # if self.rank==0:
        #     print('old', self.theCentralPlanner.informationTableMultirank)
        #     print('new', self.theCentralPlanner.globalFlows[:4])
        
    def plannerDiffusingProductionOrders(self):
        ###
        ###parallel independent operations if multirank
        ###
        self.theCentralPlanner.diffusingProductionOrders(self)
    
    def firmsProducing(self):
        self.totalProduction.append([0]*(self.rowNumber)) #for each cycle adds a sub-list of lenght number of firm class types
        self.totalCostOfProduction.append([0]*(self.rowNumber))
        self.totalCostOfUnusedFactors.append([0]*(self.rowNumber))
        self.totalInvGoodsInventories.append([0]*(self.rowNumber))
        self.totalInProgressInvGoodsInventories.append([0]*(self.rowNumber))
        self.totalConsGoodsInventories.append([0]*(self.rowNumber))
        self.totalInProgressConsGoodsInventories.append([0]*(self.rowNumber))
        self.totalLostProduction.append([0]*(self.rowNumber))
        self.totalCostOfLostProduction.append([0]*(self.rowNumber))
        self.updatedLabor.append([0]*(self.rowNumber))
        self.updatedCapital.append([0]*(self.rowNumber))
        self.totalGrossInvestmentQ.append([0]*(self.rowNumber))
        
        for aFirm in self.context.agents(agent_type=params['FIRM_TYPE']): #SHUFFLE to make them acting in random order
            aFirm.produce(self.t())
                       
            
    def plannerPreparingAndMakingDistributionOfInvGoods(self):

        self.theCentralPlanner.askFirmsInvGoodsDemand(self)
        ###
        ###sinchronize ghosts
        if self.rankNum > 1 and self.context.contains_type(agent_type=params['CEN_PLANNER_TYPE'] + 1000):
            for aReporter in self.context.agents(agent_type=params['CEN_PLANNER_TYPE'] + 1000):
                aReporter.reciveInformationLastCol()
        
        ###
        if rankNum > 1: self.context.synchronize(restore_agent)

        #test
        #if rank==0:
        #    for i in range(1,rankNum):
        #        print("from M, rank=",rank,"t=", t(), \
        #              self.theCentralPlannerReporterGhostList[i-1].invGoodsDemandList,flush=True)

        ###
        ###MULTIRANK
        ###
        #add data collected from central planner reporter of ranks > 0 to the central planner of rank 0 data
        
        self.theCentralPlanner.mergeInformationTableData()
        
        # if rank == 0:
        #     print('old1', (self.theCentralPlanner.allFirmsDesiredCapitalQsubstitutionsMultirank,
        #         self.theCentralPlanner.allFirmsRequiredCapitalQincrementMultirank,
        #         self.theCentralPlanner.allFirmsDesiredCapitalSubstitutionsMultirank,
        #         self.theCentralPlanner.allFirmsRequiredCapitalIncrementMultirank))
        #     print('new1', self.theCentralPlanner.globalFlows[4:])
            

        #determining and diffusing (if multirank) the proportionalValue to be used in the case "propotionally"
        if self.rank==0: self.theCentralPlanner.setProportionalValue(self)
        ###
        ###sinchronize ghosts
        ###
        if rankNum > 1: self.context.synchronize(restore_agent)
            
        self.theCentralPlanner.executeInvestmentGoodsDemandFromFirms(self)
    
    def firmsConcludingProduction(self):
        for aFirm in self.context.agents(params['FIRM_TYPE']):
            
            tupleOfProductionResults = aFirm.concludeProduction()

            self.totalProduction[self.t()][aFirm.sectorialClass] += tupleOfProductionResults[0]
            self.totalCostOfProduction[self.t()][aFirm.sectorialClass] += tupleOfProductionResults[1]
            self.totalCostOfUnusedFactors[self.t()][aFirm.sectorialClass] += tupleOfProductionResults[2]
            
            if not aFirm.productionType in params["investmentGoods"]: 
                self.totalConsGoodsInventories[self.t()][aFirm.sectorialClass] += tupleOfProductionResults[3]
                self.totalInProgressConsGoodsInventories[self.t()][aFirm.sectorialClass] += tupleOfProductionResults[4]  
            else: 
                self.totalInvGoodsInventories[self.t()][aFirm.sectorialClass] += tupleOfProductionResults[3]
                self.totalInProgressInvGoodsInventories[self.t()][aFirm.sectorialClass] += tupleOfProductionResults[4]
            
            #here we will need to separate invGoods and consGoods inventories (and in progr inventories)
            #same for revenues, to be added here to the series
            self.totalLostProduction[self.t()][aFirm.sectorialClass] += tupleOfProductionResults[5]
            self.totalCostOfLostProduction[self.t()][aFirm.sectorialClass] += tupleOfProductionResults[6]
            self.updatedLabor[self.t()][aFirm.sectorialClass] += tupleOfProductionResults[7]
            self.updatedCapital[self.t()][aFirm.sectorialClass] += tupleOfProductionResults[8]
            self.totalGrossInvestmentQ[self.t()][aFirm.sectorialClass] += tupleOfProductionResults[9]

    def firmsMakingFinancialTransactionsRelatedToCosts(self):
        for aBank in self.global_agent_uid.getAllAgents(agent_type=params['COMM_BANK_TYPE'], rankId=self.rank):
            if aBank[2] == self.rank:
                aBank = self.context.agent(aBank)
            else:
                aBank = self.context.ghost_agent(aBank)
            aBank.payDepositInterests()
    
    def plannerGeneratingDemandOrders(self):
        #currently independent processes, so also operating multirank
        self.theCentralPlanner.generateDemandOrders(self)
        
    def firmsMakingFinancialTransactionsRelatedToRevenues(self):
        
        for aFirm in self.context.agents(agent_type=params['FIRM_TYPE']):

            targetDeposit = aFirm.Deposits[0]
            sourceDeposit = aFirm.theCentralPlanner.Deposits[0]

            sourceDeposit.liabilityHolder.transfer(sourceDeposit, targetDeposit, aFirm.revenues)
            
        
    def bankConcludingInformation(self):
        if rankNum > 1: 
            for abankReporter in self.context.agents(agent_type=params['COMM_BANK_TYPE'] + 1000):
                abankReporter.reciveInformationLastCol()
            
            self.context.synchronize(restore_agent)

        for aBank in self.context.agents(agent_type=params['COMM_BANK_TYPE']):
            aBank.mergeInformationTableData()
        
        if rankNum > 1: 
            self.context.synchronize(restore_agent)

    def bankGetCredit(self):
        for aBank in self.context.agents(agent_type=params['COMM_BANK_TYPE']):
            advancesDemand = aBank.getAdvanceDemand()

            advances = Loan(amount=advancesDemand, interestRate=self.theCentralBank.advanceInterestRate, startTick=self.t(), length=1,
             ObservePeriod=params['loanObservationFrequency'], assetHolder=aBank, liabilityHolder=self.theCentralBank)



            aBank.Advances.append(advances)
            self.theCentralBank.Advances.append(advances)

            if len(aBank.Reserves) == 1:
                aBank.Reserves[0].amount += advancesDemand
            else:
                print('not happen')
                depositCB = Deposit(advancesDemand, aBank, self.theCentralBank)
                aBank.Reserves.append(depositCB)
                self.theCentralBank.Reserves.append(depositCB)

    def bankPayLoan(self):
        for aBank in self.context.agents(agent_type=params['COMM_BANK_TYPE']):
            aBank.payInterests(self.t())

    def creditMarket(self):
        
        demanderList = self.context.agents(agent_type=params['FIRM_TYPE'])
        supplierUids = self.global_agent_uid.getAllAgents(agent_type=params['COMM_BANK_TYPE'], rankId=self.rank)
        demanderUids = []
        for demander in demanderList:
            demanderUids.append(demander.uid)
            self.theCreditMarket.collectDemandInfo(demander.uid, [10])
        
        for supplier in self.context.agents(agent_type=params['COMM_BANK_TYPE']):
            supplier.getCreditSupply()
            self.theCreditMarket.collectSupplyInfo(supplier.uid, [supplier.loanSupply])
        
        self.theCreditMarket.collectDemander(demanderUids)
        self.theCreditMarket.collectSupplyer(supplierUids)

        
        if rankNum > 1 and self.context.contains_type(agent_type=params['CREDIT_MARKET_TYPE'] + 1000): 
            for aCerditMarketReporter in self.context.agents(agent_type=params['CREDIT_MARKET_TYPE'] + 1000):
                aCerditMarketReporter.reciveInformationLastCol()
            
        if rankNum > 1: self.context.synchronize(restore_agent)

        if rank == 0:
            self.theCreditMarket.mergeInformationTableData()
            self.theCreditMarket.excute()
        
        if rankNum > 1: self.context.synchronize(restore_agent)
        
        
        if self.theCreditMarket.matched_info != {}:
            
            for demanderUid, (supplierUid, amount) in self.theCreditMarket.matched_info.items():
                if demanderUid[2] == self.rank:
                    demander = self.context.agent(demanderUid)
                if supplierUid[2] == self.rank:
                    supplier = self.context.agent(supplierUid)
                
                loan = Loan(amount, supplier.loanInterestRate, self.t(), 5, params['loanObservationFrequency'], supplier, demander)
                
                demander.Loans.append(loan)
                supplier.Loans.append(loan)
                
                demander.Deposits[0].amount += amount
        
        
        


    def enterprisesMakingBalancesheet(self):

        
        self.totalInvGoodsRevenues.append([0]*(self.rowNumber))
        self.totalConsGoodsRevenues.append([0]*(self.rowNumber))
        
        for aFirm in self.context.agents(agent_type=params['FIRM_TYPE']):
            aFirm.makeBalancesheet(self.t())
            self.totalConsGoodsRevenues[self.t()][aFirm.sectorialClass] += aFirm.myBalancesheet[self.t(), 3]
            self.totalInvGoodsRevenues[self.t()][aFirm.sectorialClass] += aFirm.myBalancesheet[self.t(), 4]
        
        for aBank in self.context.agents(agent_type=params['COMM_BANK_TYPE']):
            aBank.makeBalancesheet(self.t())
        
        if rank == 0:
            self.theCentralPlanner.resetAttribute()
    
    
    #finish
    def finish(self):
        
        print("cpu time - calculating phase", Tc(), "rank", self.rank, flush=True)

        print(self.context.size(agent_type_ids=[params['FIRM_TYPE']]))
        
        # infos for data_analysis*.ipynb
        with open('plotInfo.csv', 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow((params["log_file_root"],rankNum,\
                             self.context.size(agent_type_ids=[params['FIRM_TYPE']])[params['FIRM_TYPE']]))
        
        
        #series____________________________________________________
        
        names=["_total_production_","_total_cost_of_production_","_total_cost_of_unused_factors_",\
               "_total_inv_goods_revenues_", "_total_cons_goods_revenues_",\
               "_total_inv_goods_inventories_","_total_in_progress_inv_goods_inventories_",\
               "_total_cons_goods_inventories_","_total_in_progress_cons_goods_inventories_",\
               "_total_lost_production_","_total_cost_of_lost_production_","_updatedLabor_","_updatedCapital_",\
               "_total_grossInvestmentQ_"]
        contents=[self.totalProduction,self.totalCostOfProduction,self.totalCostOfUnusedFactors,
                  self.totalInvGoodsRevenues, self.totalConsGoodsRevenues, 
                  self.totalInvGoodsInventories,self.totalInProgressInvGoodsInventories,
                  self.totalConsGoodsInventories,self.totalInProgressConsGoodsInventories,
                  self.totalLostProduction,self.totalCostOfLostProduction,
                  self.updatedLabor,self.updatedCapital, self.totalGrossInvestmentQ]
        
        for s in range(len(names)):
            with open(params["log_file_root"]+names[s]+str(self.rank)+'.csv', 'w', newline='') as file:
                writer = csv.writer(file)
                for k in range(params["howManyCycles"]):
                    writer.writerow(contents[s][k])

        
        #balancesheets______________________________________________
        #via pickle
        
        #creating a dictionary of firm dataframes
        #firmData={} defined in __init__
        colNames=["firm class type", "initial inventories","total costs", "revenuesCons", "revenuesInv", "consGoods inventories",\
       "invGoods inventories",  "consGoods in progr. inventories", "invGoods in progr. inventories", "profits", \
          "added value", "total production", "cost of production", "cost of unused factors", "total lost production", \
          "total cost of lost production", "cost of labor", "cost of capital", "gross investment in Q",\
            "production type"]
        

        for aFirm in self.context.agents(agent_type=params["FIRM_TYPE"]):
            self.firmData[aFirm.uid]=pd.DataFrame(aFirm.myBalancesheet)
            self.firmData[aFirm.uid].columns=colNames

        pickle.dump(self.firmData, open(params["log_file_root"]+'_balancesheetDict.p', "wb"))

        bankColNames=["total deposite", "total reserve", "total advances", "total loans", "total deposite delta", "total reserve delta"
        , "total interest deposit", "total interest advance"]
        
        
        for aBank in self.context.agents(agent_type=params["COMM_BANK_TYPE"]):
            self.bankData[aBank.uid]=pd.DataFrame(aBank.myBalancesheet)
            self.bankData[aBank.uid].columns=bankColNames

        pickle.dump(self.bankData, open(params["log_file_root"]+'_bankBalancesheetDict.p', "wb"))

        #workingMultiRank
        np.savetxt("plannerInfo.csv", self.theCentralPlanner.informationTable, delimiter=",")
        print("cpu time - finishing phase", Tc(), "rank", self.rank, flush=True)
        print("THE END!", flush=True)
    
    def start(self):
        self.runner.execute()

NameError: name 'Dict' is not defined

In [ ]:

model = Model(comm, params)
model.start()